In [1]:
import os
os.chdir(os.path.abspath(os.curdir).replace('notebooks',''))

In [2]:
import json

with open('./data/fewrel/train_wiki.json','r') as data:
    js_ls = json.load(data)

In [10]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel
  
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

model = AutoModel.from_pretrained("bert-base-uncased")

In [17]:
txt = ' '.join(js_ls['P931'][0]['tokens'])

tokenizer(txt).tokens()

['[CLS]',
 'mer',
 '##pati',
 'flight',
 '106',
 'departed',
 'jakarta',
 '(',
 'c',
 '##g',
 '##k',
 ')',
 'on',
 'a',
 'domestic',
 'flight',
 'to',
 'tan',
 '##jun',
 '##g',
 'panda',
 '##n',
 '(',
 't',
 '##j',
 '##q',
 ')',
 '.',
 '[SEP]']

In [ ]:
from source

In [15]:
txt

'Merpati flight 106 departed Jakarta ( CGK ) on a domestic flight to Tanjung Pandan ( TJQ ) .'

In [8]:
print(js_ls['P931'][0])

{'tokens': ['Merpati', 'flight', '106', 'departed', 'Jakarta', '(', 'CGK', ')', 'on', 'a', 'domestic', 'flight', 'to', 'Tanjung', 'Pandan', '(', 'TJQ', ')', '.'], 'h': ['tjq', 'Q1331049', [[16]]], 't': ['tanjung pandan', 'Q3056359', [[13, 14]]]}
